# Welcome to Intro to Artificial Inteligence assignment 1 practical part!

In this part you will get to implement algotithms learned in class from scratch.


The pupose of this notebook is to provide the requirments of the practical part and some visualizations. Your implementation should be in the provided Algorithms.py file.

**Important tip:** If the same variable name appears in two different cells, the variable value will be determined by the last cell to run, rather than by the position of the cell. Let's see an example,  run the three cells below:

In [1]:
# cell 2
x = 5

now rerun cell 1 and print x again.

The same applies to functions, classes, etc.


In [2]:
import time
from IPython.display import clear_output

from DragonBallEnv import DragonBallEnv
from typing import List, Tuple
from Algorithms import *


# Maps
A map can be produced manually as shown in the cell below. We will only work on maps in which there is a route from the initial state to a final state.


# The Dragon Ball Environment ⛷
The file `DragonBallEnv.py` implements our own version of the dragon ball environment. It is recommended to go through the code. 

**Note: You are not allowed to change this file.**

In [3]:
env = DragonBallEnv(MAPS["8x8"])
state = env.reset()
print('Initial state:', state)
print('Goal states:', env.goals) 

NameError: name 'MAPS' is not defined

In [ ]:
print(f"Action Space {env.action_space}")
print(f"State Space {env.observation_space}")

Now we will go through some usfel methods (It is still recommended to go through the other methods in the class):

`render()` - returns a printable view of the board.

The pink square represents the agent. The letter ״*H*״ represents holes, and the yellow square are dragon balls or goal.

Here are two more useful methods:

`get_state()` - Returns the current state of the agent.

`set_state(state)` - Sets the current state of the agent.

`succ(state)` - Returns a dictionary that contains information on all the successors of a state.

*   The keys are the actions.
*   The values are tuples of the form (next state, cost, terminated). Note that terminated is true when the agent reaches a **final state** or a **hole**.



***Tip***: You can loop through both keys and values by using the `items()` method.


As you can see, the action 0 (down) will move your agent to state 26 and the transition will cost you 10. Action 1 will move your agent to state 19, which is a hole that will terminate your run.

`is_final_state(state)` can assist you in distinguishing between a final state and a hole.

Let's see what happens when we apply succ(state) on a hole:



As you can see, if the operator cannot be applied to the state, all returned values are "None"

In [ ]:
new_state, cost, terminated = env.step(DOWN)
print(env.render())
print("New state:", new_state)
print("cost:", cost)
print("Terminated:", terminated)

We've finished our demo 🥳 and it's time to reset the environment.

In [ ]:
print(f"current state befor reset: {env.get_state()}")
env.reset()
print(f"current state after reset: {env.get_state()}")

In [ ]:
class RandomAgent():
  def __init__(self):
    self.env = None

  def animation(self, epochs: int ,state: int, action: List[int], total_cost: int) -> None:
      clear_output(wait=True)
      print(self.env.render())
      print(f"Timestep: {epochs}")
      print(f"State: {state}")
      print(f"Action: {action}")
      print(f"Total Cost: {total_cost}")
      time.sleep(1)

  def random_search(self, DragonBallEnv: env) -> Tuple[List[int],int]:
    self.env = env
    self.env.reset()
    epochs = 0
    cost = 0
    total_cost = 0

    actions = []

    state = self.env.get_initial_state()
    while not self.env.is_final_state(state):
      action = self.env.action_space.sample()
      new_state, cost, terminated = self.env.step(action)
        
      while terminated is True and self.env.is_final_state(state) is False:
        self.env.set_state(state)
        action = self.env.action_space.sample()
        new_state, cost, terminated = self.env.step(action)
        
      actions.append(action)
      total_cost += cost
      state = new_state
      epochs += 1
      
      self.animation(epochs,state,action,total_cost)

    return (actions, total_cost)

In [ ]:
# agent = RandomAgent()
# agent.random_search(env)

As you can see, a random policy is, unsurprisingly, not a good policy. However, what else can we do?

This is where you come in!

In this assignment you will be required to implement the following algorithms taught in class in order to solve the problem.

Algorithms: 
1. BFS-G
2. W-A*
3. epsilon-A*

Important to note!

Each agent should return a tuple: (actions, cost, expended) 
*  actions - the list of integers containing the sequence of actions that produce your agent's solution (and not the entire search process).
* cost -  an integer which holds the total cost of the solution.
* expanded - an integer which holds the number of nodes that have been expanded during the search (A node is considered expanded if we check for it's successors).

The solution to our search problem is the a to the final state, not the final state itself. By saving the actions, we are able to restore the path your agent found.


Any other output, unless otherwise specified, will cause the running of the tests to fail and will result in a grade of 0 !



In [ ]:
def print_solution(actions,env: DragonBallEnv) -> None:
    env.reset()
    total_cost = 0
    print(env.render())
    print(f"Timestep: {1}")
    print(f"State: {env.get_state()}")
    print(f"Action: {None}")
    print(f"Cost: {0}")
    time.sleep(1)

    for i, action in enumerate(actions):
      state, cost, terminated = env.step(action)
      total_cost += cost
      clear_output(wait=True)

      print(env.render())
      print(f"Timestep: {i + 2}")
      print(f"State: {state}")
      print(f"Action: {action}")
      print(f"Cost: {cost}")
      print(f"Total cost: {total_cost}")
      
      time.sleep(1)

      if terminated is True:
        break

Now lets test your BFS agent!

In [ ]:
%load_ext autoreload
%autoreload 2

In [5]:

# all the imports and equals lines in one cell
# import importlib

import time
from IPython.display import clear_output
from DragonBallEnv import DragonBallEnv
from typing import List, Tuple
from Algorithms import *
# importlib.reload(BFSAgent)
DOWN = 0
RIGHT = 1
UP = 2
LEFT = 3
MAPS = {
    "4x4": ["SFFF",
            "FDFF",
            "FFFD",
            "FFFG"],
    "8x8": [
        "SFFFFFFF",
        "FFFFFTAL",
        "TFFHFFTF",
        "FFFFFHTF",
        "FAFHFFFF",
        "FHHFFFHF",
        "DFTFHDTL",
        "FLFHFFFG",
    ],
}
env = DragonBallEnv(MAPS["8x8"])
state = env.reset()

BFS_agent = BFSAgent()
actions, total_cost, expanded = BFS_agent.search(env)
print(f"Total_cost: {total_cost}")
print(f"Expanded: {expanded}")
print(f"Actions: {actions}")

assert total_cost == 119.0, "Error in total cost returned"

(0, False, False)
(8, False, False)
(1, False, False)
(16, False, False)
(9, False, False)
(2, False, False)
(24, False, False)
(17, False, False)
(10, False, False)
(3, False, False)
(32, False, False)
(25, False, False)
(18, False, False)
(11, False, False)
(4, False, False)
(40, False, False)
(33, False, False)
(26, False, False)
(19, False, False)


TypeError: 'NoneType' object is not subscriptable

## 2. Weighted A*
TO DO: implement Wighted A* like shown in class.

Note:

*   A parameter called `h_weight` is passed to `Greedy_Best_First_search()`, which indicates how much weight is given to the heuristics (ranging from 0 to 1).
*   The heurisitcs needed to be implemented. Instructions in dry pdf.






In [ ]:
WA_agent = WeightedAStarAgent()
actions, total_cost, expanded = WA_agent.search(env, h_weight=0.5)
print(f"Total_cost: {total_cost}")
print(f"Expanded: {expanded}")
print(f"Actions: {actions}")

## 3. A*-epsilon::
TO DO: implement A*-epsilon: like shown in class.

use the same heuristic as in previous sections.

Note:
*   A parameter called `epsilon` is passed to `A_star_epsilon_search()`.
*   We will not test the amount of expanded nodes for this algorithm.


In [ ]:
print_solution(actions, env)

In [ ]:
import csv

test_boards = {
"map12x12": 
['SFAFTFFTHHHF',
'AFLTFFFFTALF',
'LHHLLHHLFTHD',
'HALTHAHHADHF',
'FFFTFHFFAHFL',
'LLTHFFFAHFAT',
'HAAFFALHTATF',
'LLLFHFFHTLFH',
'FATAFHTTFFAF',
'HHFLHALLFTLF',
'FFAFFTTAFAAL',
'TAAFFFHAFHFG'],
"map15x15": 
['SFTTFFHHHHLFATF',
'ALHTLHFTLLFTHHF',
'FTTFHHHAHHFAHTF',
'LFHTFTALTAAFLLH',
'FTFFAFLFFLFHTFF',
'LTAFTHFLHTHHLLA',
'TFFFAHHFFAHHHFF',
'TTFFLFHAHFFTLFD',
'TFHLHTFFHAAHFHF',
'HHAATLHFFLFFHLH',
'FLFHHAALLHLHHAT',
'TLHFFLTHFTTFTTF',
'AFLTDAFTLHFHFFF',
'FFTFHFLTAFLHTLA',
'HTFATLTFHLFHFAG'],
"map20x20" : 
['SFFLHFHTALHLFATAHTHT',
'HFTTLLAHFTAFAAHHTLFH',
'HHTFFFHAFFFFAFFTHHHT',
'TTAFHTFHTHHLAHHAALLF',
'HLALHFFTHAHHAFFLFHTF',
'AFTAFTFLFTTTFTLLTHDF',
'LFHFFAAHFLHAHHFHFALA',
'AFTFFLTFLFTAFFLTFAHH',
'HTTLFTHLTFAFFLAFHFTF',
'LLALFHFAHFAALHFTFHTF',
'LFFFAAFLFFFFHFLFFAFH',
'THHTTFAFLATFATFTHLLL',
'HHHAFFFATLLALFAHTHLL',
'HLFFFFHFFLAAFTFFDAFH',
'HTLFTHFFLTHLHHLHFTFH',
'AFTTLHLFFLHTFFAHLAFT',
'HAATLHFFFHHHHAFFFHLH',
'FHFLLLFHLFFLFTFFHAFL',
'LHTFLTLTFATFAFAFHAAF',
'FTFFFFFLFTHFTFLTLHFG']}

test_envs = {}
for board_name, board in test_boards.items():
    test_envs[board_name] = DragonBallEnv(board)


BFS_agent = BFSAgent()
WAStar_agent = WeightedAStarAgent()

weights = [0.5, 0.7, 0.9]

agents_search_function = [
    BFS_agent.search,
]

header = ['map',  "BFS-G cost",  "BFS-G expanded",\
           'WA* (0.5) cost', 'WA* (0.5) expanded', 'WA* (0.7) cost', 'WA* (0.7) expanded', 'WA* (0.9) cost', 'WA* (0.9) expanded']

with open("results.csv", 'w') as f:
  writer = csv.writer(f)
  writer.writerow(header)
  for env_name, env in test_envs.items():
    data = [env_name]
    for agent in agents_search_function:
      _, total_cost, expanded = agent(env)
      data += [total_cost, expanded]
    for w in weights:
        _, total_cost, expanded = WAStar_agent.search(env, w)
        data += [total_cost, expanded]

    writer.writerow(data)